# Projects: Looking Back, Looking Forward

## Project 1 is graded

- The TA will return grades to you shortly
- I have put notes in your notebooks and source code, you can merge them in you'd like


### I was generally pleased

- Very few issues with correctness
- Everyone got at least ~2x speed up
  - Everyone gets at least 5 speed points
  - **But why stop there?**

![runtimes](runtimes.png)

![speedup](speedups.png)

Clearly much greater speedup than 2x is possible.

### Here is a question I wish I had included as part of the assignment that I will include next year

- **Predictions (X pts)**: for each of the benchmarks, predict a **lower bound on the runtime for all submissions**.  Explain how you arrived at your prediction.

### How would I go about answering that?

We could model the runtime as $N_t$ times the runtime of one time step.

Even i fyou changed the structure of your code, at each time step you have to compute two updates to a particles position: the noise and the acceleration.

$$ T = T_{accel} + T_{noise} $$

### $ T_{noise} $:

Noise is a streaming kernel.  As such, a simple model for the runtime is the maximum of its compute and memory components.

$$ T_{noise} = \max\{T_{noise,compute} , T_{noise,mem}\} $$

$ T_{noise,mem} $ consists of one update per particle, so it consists of one load and one store for each
particle coordinate, $T_{noise,mem} \geq 2 * b * 3 * N_p * 8$, were $b$ is the $bytes / sec$ bandwidth of the node
and $N_p$ is the number of particles (and coordinates are doubles, which are 8 bytes).

Note that there is no number of threads in this calculation!  We simply want one number about the maching $b$: the maximum bandwidth at which we can do a load/store $3 * N_p * 8$ bytes.

If this number of bytes fills the cache, we should use the bandwidth from main memory; if not, we can still get a lower bound by measuring the peak STREAMS Scale benchmark for the same amount of bytes.

$ T_{noise,compute} $ consists of generating three Gaussian random variables per particle.  I didn't show you what this algorithm is, but much like Prof. Vuduc's optimizations for the compute kernel in the [FMM example](http://vuduc.org/cse6230/slides/HPCA-tutorial-part-2.pdf), we can apply this kernel in place with no memory movement to estimate its rate on on core.  We can then multiply by the number of cores on the node to get the maximum rate for the node, and then divide $N_p$ by this great to get the minimum time.

### $ T_{accel} $:

From a compute-centric viewpoint, `IXGetPairs()` is all overhead.  The real work of `accelerate()` is the update to the velocity vector, once we already know what the pairs are.  The amount of "real work" is thus the time to compute the interaction and write the update for each pair.

$$ T_{accel} = \max\{T_{accel,compute}, T_{accel,mem}\} $$

$T_{accel,mem}$ consists of writing two updates per pair, so it consists of one load and one store for each
velocity component for two particles.  However, if we somehow had perfect caching, each velocity component could be loaded once, completed updated by all its interactions and then stored.  A truly optimistic lower bound would thus be the same as streaming the velocity components, $T_{accel,mem} \geq T_{noise,mem}$.  To come up with a tighter lower bound, we would have to say something about the caching behavior of the method.

$T_{accel,compute}$ consists of compute one interaction per pair.  Let the number of pairs be $N_{pairs}$:  how does $N_{pairs}$ compare to $N_p$?  It is probabilistic and changing, but in the code, I already computed for you what the expected number of pairs would be for each time step.

Once we esimate $N_{pairs}$, the time to compute one pair interaction can be estimated in the same way that the time to compute one noisy update can be estimated.  Some of you noticed that `remainder()`, a part of computing the periodic displacement, is a significant portion of the runtime, so it would have been worthwhile to get an estimate on just how fast this operation is in isolation.

### What about `IXGetPairs()`?

Our super optimistic performance model above assumes that we get pairs for free.  This is of course not true: `IXGetPairs()` is a significant portion of the runtime.  So how should we judge when an implementation of `IXGetPairs()` is good enough?  When can we stop optimizing?

One thing to note is that the binning strategy is meant to limit the amount of comparisons that happen between particles.  **If the runtime of `IXGetPairs()` is not scaling with the number of particles found, then chances are it is being inefficient.**

## Common Mistakes in Project 1

1. **Leaving `boxdim` unchanged!** If you are not changing `boxdim` with `L`, you have an $O(N_p^2)$ method!

```
  if (use_ix) {
    int boxdim = 4; /* how could we choose boxdim ? */
    int maxNx = Np; /* how should we estimate the maximum number of interactions? */
    err = IXCreate(L, boxdim, maxNx, &(a->ix));CHK(err);
  }
```

2. Parallelizing only the outermost box loop!  `#pragma omp parallel for collapse(3)`.  Your thread parallelism will be limited by `boxdim` (which, if you did not change it, will be limited by 4)

```
  #pragma omp parallel for
  for (idx=0; idx<boxdim; idx++)
  {
    for (idy=0; idy<boxdim; idy++)
    {
      for (idz=0; idz<boxdim; idz++)
      {
        /* ... */
      }
    }
  }
```

3. False sharing

```
   ix->threadPairCounts[thread_id++]; /* threadPairCounts is densely packed into cache lines
```

4. Small errors about the calculation of when an upperbound $R$ on the interaction radius is invalidated by the maximum displacements.

5. Missing `#pragma omp critical` or `#pragma omp atomic`